This function will automatically extract all the frames and send 1/10 frames into the `val` folder. 

To get enough data for one class, I will record a video of 1 mins at the rate of 30 fps. 
It will allow me to have toally 30*60 = 1800 images to works on. 
I will set the interval as 2 so that I have totally 900 images to work with. 


The toall images for a class will be about 1.5G, for 4 classes, I will need 6 G space, which should be good. 

In [1]:
import cv2
import os

def extract_frames(video_path, class_name, output_dir, interval=1):
    # Open the video file using OpenCV
    video = cv2.VideoCapture(video_path)
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Loop through the frames and extract every nth frame
    frame_count = 2000 # reset the value,  
    frame_count_val = 2000 # this value determin the 
  
    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        if not ret:
            print("End of frames")
            break
        
        # Extract frames at the specified interval
        if frame_count % interval == 0:
            # put one of ten images into 'test' folder
            if frame_count_val % 8 == 0:
                output_folder = os.path.join(output_dir, 'val', class_name)
                output_path = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
            else: # Construct the output filename
                output_folder = os.path.join(output_dir, 'train', class_name)
                output_path = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
            
            frame_count_val += 1
            # Resize the frame to a fixed size if necessary
            # frame = cv2.resize(frame, (224, 224))
            
            # see if folder exist, create one
            os.makedirs(output_folder, exist_ok=True)
            
            # Save the frame as an image file
            cv2.imwrite(output_path, frame)
            #print(output_path)

        frame_count += 1
    
    # Release the video file and close the output file
    video.release()



The code below will loop through the four videos in the `video` folder

This file should be in the folder called `calssification`
Here is the structure of the folder:

``````


classification   
  - Extract_frames-3-15-2023.ipynb
  - videos
    - task1
       - class1.mov
       - class2.mov
       ....
    - task2
       - class1.mov
       - class2.mov
       ....
       
  - data  
    - task1
       - train   
           - class1  
           - class2 
       - val  
           - class1  
           - class2  
    - task2
           - class1  
           - class2 
       - val  
           - class1  
           - class2  
``````
      

In [4]:
#video_names_list = ['carpet', 'lab_white', 'corridor_white', 'hall_red']
#video_names_list = ['carpet', 'lab_white', 'corridor_white', 'hall_red']
import os
path = '../videos/fake3'
for video_path in os.listdir(path): ## all the videos in the directory
    video_name = os.path.splitext(video_path)[0]   
    class_name = video_name
    output_dir = '../data/fake'
    # obtain the relative path to open the file. 
    video_path = os.path.join(path,video_path)
    # call the function to extract the frames. 
    extract_frames(video_path, class_name, output_dir, interval=1) 
    print('Finish one class')
    # verify the path with the `print`
    #print(f'Here are th path: {video_path}, {class_name}, {output_dir}')

    

NameError: name 'cv2_cuda_GpuMat' is not defined

Use GPU to speed up the processing
---

This code does not work. 

In [3]:
import cv2
import os
import cv2.cuda as cv2_cuda

def extract_frames(video_path, class_name, output_dir, interval=1):
    # Open the video file using OpenCV
    video = cv2.VideoCapture(video_path)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create CUDA versions of OpenCV objects
    gpu_frame = cv2_cuda_GpuMat()
    gpu_resized_frame = cv2_cuda_GpuMat()

    # Loop through the frames and extract every nth frame
    frame_count = 3700
    frame_count_val = 3700

    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        if not ret:
            print("End of frames")
            break

        # Extract frames at the specified interval
        if frame_count % interval == 0:
            # put one of ten images into 'test' folder
            if frame_count_val % 8 == 0:
                output_folder = os.path.join(output_dir, 'val', class_name)
                output_path = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
            else:
                output_folder = os.path.join(output_dir, 'train', class_name)
                output_path = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')

            frame_count_val += 1

            # Upload frame to GPU memory
            gpu_frame.upload(frame)

            # Resize the frame on the GPU to a fixed size if necessary
            # gpu_resized_frame = cv2_cuda.resize(gpu_frame, (224, 224))

            # Download the frame back to CPU memory
            # resized_frame = gpu_resized_frame.download()
            
            # If resizing is not necessary, download the frame directly
            resized_frame = gpu_frame.download()

            # Create the output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

            # Save the frame as an image file
            cv2.imwrite(output_path, resized_frame)
            # print(output_path)

        frame_count += 1

    # Release the video file and close the output file
    video.release()


In [2]:
import cv2
import os
import time

def extract_frames(video_path, class_name, output_dir, interval=1):
    # Open the video file using OpenCV
    video = cv2.VideoCapture(video_path)

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Loop through the frames and extract every nth frame
    frame_count = 3500
    frame_count_val = 3500
  
    io_start_time = time.time()  # Start measuring file I/O time

    while True:
        # Read the next frame from the video
        ret, frame = video.read()
        if not ret:
            print("End of frames")
            break
        
        # Extract frames at the specified interval
        if frame_count % interval == 0:
            # put one of ten images into 'test' folder
            if frame_count_val % 8 == 0:
                output_folder = os.path.join(output_dir, 'val', class_name)
                output_path = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
            else:
                output_folder = os.path.join(output_dir, 'train', class_name)
                output_path = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
            
            frame_count_val += 1
            
            # Save the frame as an image file
            io_start = time.time()  # Start measuring individual file I/O time
            cv2.imwrite(output_path, frame)
            io_end = time.time()  # End measuring individual file I/O time
            io_duration = io_end - io_start

            print(f"Frame {frame_count}: File I/O time: {io_duration:.4f} seconds")

        frame_count += 1
    
    io_end_time = time.time()  # End measuring file I/O time
    io_duration_total = io_end_time - io_start_time

    print(f"Total File I/O time: {io_duration_total:.4f} seconds")

    # Release the video file
    video.release()
